# Price Data Processing

## Analyze

Loading data...

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

DATA_FOLDER = "data/recordings/poloniex/"

csvs = [f for f in os.listdir(DATA_FOLDER) if f.endswith(".csv")]

all_series = dict()
for csv in csvs:
    p = os.path.join(DATA_FOLDER, csv)
    s = pd.read_csv(p, index_col='timestamp')
    s.index = pd.to_datetime(s.index, unit='s')
    all_series[csv[:-4]] = s

Print histograms for all high, low and closing prices

In [7]:
for n in all_series:
    s = all_series[n]
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 3, 1).set_title(n)
    _ = s["high"].hist()
    plt.subplot(1, 3, 2).set_title("low")
    _ = s["low"].hist()
    plt.subplot(1, 3, 3).set_title("close")
    _ = s["close"].hist()
    plt.show()

Any missing data?

In [14]:
for n in all_series:
    print(all_series[n].isna().where(lambda x: x == True).count())

open               0
high               0
low                0
close              0
volume             0
quoteVolume        0
weightedAverage    0
dtype: int64
open               0
high               0
low                0
close              0
volume             0
quoteVolume        0
weightedAverage    0
dtype: int64
open               0
high               0
low                0
close              0
volume             0
quoteVolume        0
weightedAverage    0
dtype: int64
open               0
high               0
low                0
close              0
volume             0
quoteVolume        0
weightedAverage    0
dtype: int64
open               0
high               0
low                0
close              0
volume             0
quoteVolume        0
weightedAverage    0
dtype: int64
open               0
high               0
low                0
close              0
volume             0
quoteVolume        0
weightedAverage    0
dtype: int64
open               0
high               

### Problems
Found the following problematic data sets:
* BTC_ZEC seems to have a unreasonable large value at the start
* BTC_GNT has a very high "high" price of 0.02 somewhere in the data set
* BTC_ETH has a shifted histogram of the "high" prices

Describing data sets

In [9]:
all_series["BTC_ZEC"].describe()

,timestamp,open,high,low,close,volume,quoteVolume,weightedAverage
count,3.201300e+04,32013.000000,32013.000000,32013.000000,32013.000000,32013.000000,32013.000000,32013.000000
mean,1.506488e+09,0.229043,0.326667,0.090594,0.150861,19.762768,296.837116,0.123018
std,1.663470e+07,16.545349,24.983040,1.530495,8.907074,68.360628,755.722842,5.248673
min,1.477678e+09,0.016570,0.017062,0.016559,0.016570,0.000000,0.000000,0.016735
25%,1.492083e+09,0.030934,0.031071,0.030813,0.030936,2.250785,57.211432,0.030950
50%,1.506488e+09,0.042590,0.042940,0.042296,0.042600,6.873579,136.889102,0.042609
75%,1.520894e+09,0.059337,0.059930,0.059000,0.059385,18.192289,313.181690,0.059393
max,1.535299e+09,2500.000000,3300.000000,260.000000,1510.000000,5510.903511,48779.018015,906.552755


### Possible Solutions:
* BTC_ZEC: extreme high prices at the start, consider replacing with different coin
* BTC_GNT: one value in "high" is 0.02 -> remove
* BTC_ETH: one value in "high" is 50 -> remove

## Processing Data

Implementing solutions and preparing the data.

In [15]:
PROCESSED_DIR = "processed"
processed_path = os.path.join(DATA_FOLDER, PROCESSED_DIR)

#### BTC_ZEC

In [16]:
all_series["BTC_ZEC"]["high"] = all_series["BTC_ZEC"]["high"].apply(lambda x: None if x > 100 else x)
all_series["BTC_ZEC"]["low"] = all_series["BTC_ZEC"]["low"].apply(lambda x: None if x > 100 else x)
all_series["BTC_ZEC"]["close"] = all_series["BTC_ZEC"]["close"].apply(lambda x: None if x > 100 else x)
all_series["BTC_ZEC"].bfill()
all_series["BTC_ZEC"].describe()

,open,high,low,close,volume,quoteVolume,weightedAverage
count,32013.000000,31731.000000,31743.000000,31739.000000,32013.000000,32013.000000,32013.000000
mean,0.229043,0.056571,0.055787,0.056152,19.762768,296.837116,0.123018
std,16.545349,0.063738,0.062999,0.063458,68.360628,755.722842,5.248673
min,0.016570,0.017062,0.016559,0.016570,0.000000,0.000000,0.016735
25%,0.030934,0.031015,0.030761,0.030880,2.250785,57.211432,0.030950
50%,0.042590,0.042633,0.042082,0.042330,6.873579,136.889102,0.042609
75%,0.059337,0.059200,0.058466,0.058762,18.192289,313.181690,0.059393
max,2500.000000,0.999900,0.995000,0.989000,5510.903511,48779.018015,906.552755


#### BTC_GNT

In [17]:
all_series["BTC_GNT"]["high"] = all_series["BTC_GNT"]["high"].\
    apply(lambda x: None if x > 0.01 else x)
all_series["BTC_GNT"] = all_series["BTC_GNT"].bfill()
all_series["BTC_GNT"].describe()

,open,high,low,close,volume,quoteVolume,weightedAverage
count,53495.000000,24783.000000,53495.000000,53495.000000,53495.000000,53495.000000,53495.000000
mean,0.040753,0.007620,0.039587,0.039818,273.511039,10686.633361,0.039813
std,0.218386,0.003218,0.031976,0.032147,592.132751,22243.997893,0.032142
min,0.001470,0.001575,0.001370,0.001470,0.000000,0.000000,0.001441
25%,0.012088,0.003928,0.012005,0.012097,20.791485,941.110889,0.012084
50%,0.026802,0.009985,0.026550,0.026811,81.596487,3561.118455,0.026823
75%,0.069660,0.009985,0.069396,0.069643,262.291944,10590.330798,0.069632
max,50.000000,0.010000,0.148300,0.150200,17839.228595,603486.652446,0.150363


#### BTC_ETH

In [18]:
all_series["BTC_ETH"]["high"] = all_series["BTC_ETH"]["high"].\
    apply(lambda x: None if x > 1 else x)
all_series["BTC_ETH"] = all_series["BTC_ETH"].bfill()
all_series["BTC_ETH"].describe()

,open,high,low,close,volume,quoteVolume,weightedAverage
count,26618.000000,26618.000000,26618.000000,26618.000000,26618.000000,2.661800e+04,26618.000000
mean,0.000069,0.000069,0.000067,0.000068,13.168069,1.456714e+05,0.000068
std,0.000132,0.000132,0.000049,0.000049,46.122418,4.369128e+05,0.000049
min,0.000015,0.000015,0.000015,0.000015,0.000000,0.000000e+00,0.000015
25%,0.000033,0.000034,0.000033,0.000033,0.390655,9.354299e+03,0.000033
50%,0.000055,0.000055,0.000054,0.000055,1.807242,3.861787e+04,0.000055
75%,0.000076,0.000077,0.000076,0.000076,8.959891,1.266713e+05,0.000076
max,0.020000,0.020000,0.000272,0.000275,2613.749372,2.768122e+07,0.000275


### Alignment

Truncate to common last time stamp

In [19]:
last_ts = datetime.datetime.now()
for n in all_series:
    if all_series[n].index.max() < last_ts:
        last_ts = all_series[n].index.max() 
        
for n in all_series:
    all_series[n] = all_series[n][:last_ts]

Align all date frames to common length

In [20]:
max_len = 0
longest_series = "None found!"
for n in all_series:
    l = all_series[n].count()[0]
    if l > max_len:
        max_len = l
        longest_series = n

for n in all_series:
    l = all_series[n].count()[0]
    if l < max_len:
        all_series[n] = all_series[n].align(all_series[longest_series], method='bfill')[0]
        
print("Max len: {0}".format(max_len))
print(longest_series)

Max len: 55333
BTC_DASH


Print counts and make sure they are all the same

In [21]:
for n in all_series:
    cnt = all_series[n].count()[0]
    print(cnt)
    print(cnt == max_len)
    
    ts = all_series[n].index.max()
    print(ts)
    print(ts == last_ts)

55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True
55333
True
2018-08-26 16:00:00
True


### Export

In [23]:
for n in all_series:
    all_series[n].to_csv(os.path.join(processed_path, (n + ".csv")))